In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession, Row, Window
import pandas as pd
from collections import OrderedDict
import json, xmltodict
import pyspark.sql.functions as F
import os

In [2]:
# Create new Sparkcontext instance for analysis
findspark.init()
sc = pyspark.SparkContext(appName="FB_Messenger_Analysis", master='local[*]')
spark = SparkSession.builder.getOrCreate()

In [3]:
fb_messages_all_df = spark.read.option("header",True).csv(r"C:\Users\RajNa\Documents\GitHub\Pyspark_Message_Analysis_v4\Pyspark_Message_Analysis_v4\FB_Parsed_Messages.csv")

In [4]:
# Set the date field (hoping to move this to the Load file once I stop hitting OOM errors)

spark.conf.set("spark.sql.session.timeZone", "America/New_York")
fb_messages_all_df = fb_messages_all_df.withColumn('timestamp_date', F.to_date(F.from_unixtime(fb_messages_all_df['timestamp_ms'] / 1000)))
fb_messages_all_df = fb_messages_all_df.withColumn('timestamp_datetime', F.from_unixtime(fb_messages_all_df['timestamp_ms'] / 1000))
fb_messages_all_df = fb_messages_all_df.withColumn('timestamp_year', F.year(F.col('timestamp_date')))
spark.conf.unset("spark.sql.session.timeZone")

In [ ]:
# Who have I messaged the most overall?


fb_messages_all_count_df = fb_messages_all_df.groupBy('sender_name').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_sender_count_df = fb_messages_all_count_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(25)


In [ ]:
# Who have I messaged the most overall, by year, filtering out myself, across all chats

my_name = 'Raj Narayan'

fb_messages_all_count_year_df = fb_messages_all_df.filter(fb_messages_all_df.sender_name != my_name) \
.groupBy('sender_name', 'timestamp_year').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_sender_count_year_df = fb_messages_all_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


In [5]:
# How many messages do I send by year?

my_name = 'Raj Narayan'

fb_messages_me_count_year_df = fb_messages_all_df.filter(fb_messages_all_df.sender_name == my_name).groupBy('sender_name', 'timestamp_year').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_me_count_year_df = fb_messages_me_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


+-----------+--------------+-----+-----------+
|sender_name|timestamp_year|count|rank_sender|
+-----------+--------------+-----+-----------+
|Raj Narayan|          2019|34742|          1|
|Raj Narayan|          2017|34147|          2|
|Raj Narayan|          2018|30896|          3|
|Raj Narayan|          2020|25471|          4|
|Raj Narayan|          2016|13603|          5|
|Raj Narayan|          2021|12767|          6|
|Raj Narayan|          2015| 8266|          7|
|Raj Narayan|          2012| 5359|          8|
|Raj Narayan|          2011| 4150|          9|
|Raj Narayan|          2013| 2416|         10|
|Raj Narayan|          2014| 1808|         11|
|Raj Narayan|          2010|   67|         12|
|Raj Narayan|          2009|    2|         13|
+-----------+--------------+-----+-----------+



In [ ]:
# How many messages do I send by year by type?

my_name = 'Raj Narayan'

fb_messages_me_type_count_year_df = fb_messages_all_df \
.filter(fb_messages_all_df.sender_name == my_name) \
.groupBy('sender_name', 'timestamp_year', 'thread_type').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_me_type_count_year_df = fb_messages_me_type_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


In [ ]:
# Who have I messaged the most overall, by year, filtering out myself, across all chats by type

my_name = 'Raj Narayan'

fb_messages_all_type_count_year_df = fb_messages_all_df.filter(fb_messages_all_df.sender_name != my_name) \
.groupBy('sender_name', 'timestamp_year','thread_type').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_sender_type_count_year_df = fb_messages_all_type_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


In [ ]:
# Who have I messaged the most overall, by year, filtering out myself and group chats

my_name = 'Raj Narayan'

fb_messages_regular_count_year_df = fb_messages_all_df.filter(fb_messages_all_df.sender_name != my_name) \
.filter(fb_messages_all_df.thread_type != 'RegularGroup') \
.groupBy('sender_name', 'timestamp_year','thread_type').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_regular_count_year_df = fb_messages_regular_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


In [ ]:
# Who have I messaged the most overall, by year, filtering out myself and non-group chats

my_name = 'Raj Narayan'

fb_messages_regular_group_count_year_df = fb_messages_all_df.filter(fb_messages_all_df.sender_name != my_name) \
.filter(fb_messages_all_df.thread_type != 'Regular') \
.groupBy('sender_name', 'timestamp_year','thread_type').count()

countWindowSpec = Window.partitionBy().orderBy(F.col('count').desc())

fb_window_regular_group_count_year_df = fb_messages_regular_group_count_year_df.withColumn('rank_sender', F.row_number().over(countWindowSpec)).show(50)


In [ ]:
# Who have I messaged the most overall, by year, filtering out myself, across all chats, ranked by 2021

my_name = 'Raj Narayan'

fb_messages_all_count_year_pivot_df = fb_messages_all_df \
.filter(fb_messages_all_df.sender_name != my_name) \
.filter(fb_messages_all_df.timestamp_year.isNotNull()) \
.groupBy('sender_name').pivot('timestamp_year').count()

orderBy2021WindowSpec = Window.partitionBy().orderBy(F.col('2021').desc())
orderBy2020WindowSpec = Window.partitionBy().orderBy(F.col('2020').desc())
orderBy2019WindowSpec = Window.partitionBy().orderBy(F.col('2019').desc())

fb_messages_all_count_year_pivot_df \
.withColumn('2019_rank', F.row_number().over(orderBy2019WindowSpec))\
.withColumn('2020_rank', F.row_number().over(orderBy2020WindowSpec))\
.withColumn('2021_rank', F.row_number().over(orderBy2021WindowSpec))\
.show(1000)

In [ ]:
# Who have I messaged the most overall, by year, filtering out myself, across regular chats, ranked by 2021

my_name = 'Raj Narayan'

fb_messages_all_count_year_pivot_df = fb_messages_all_df \
.filter(fb_messages_all_df.sender_name != my_name) \
.filter(fb_messages_all_df.timestamp_year.isNotNull()) \
.filter(fb_messages_all_df.thread_type == 'Regular') \
.groupBy('sender_name').pivot('timestamp_year').count()

orderBy2021WindowSpec = Window.partitionBy().orderBy(F.col('2021').desc())
orderBy2020WindowSpec = Window.partitionBy().orderBy(F.col('2020').desc())
orderBy2019WindowSpec = Window.partitionBy().orderBy(F.col('2019').desc())

fb_messages_all_count_year_pivot_df \
.withColumn('2019_rank', F.row_number().over(orderBy2019WindowSpec))\
.withColumn('2020_rank', F.row_number().over(orderBy2020WindowSpec))\
.withColumn('2021_rank', F.row_number().over(orderBy2021WindowSpec))\
.show(1000)